In [2]:
import tskit, msprime
import numpy as np
import pandas as pd

In [6]:
def comparison_within(p1):
    num_sites = p1.shape[0]
    num_comparisons = p1.shape[1]
    result = []
    for i in range(num_comparisons):
        p2 = np.roll(p1,1, axis=1)
        result.append(np.sum(np.mod(p1+p2, 2))/num_sites)
    return result

In [20]:
def comparison_between(p1,p2):
    num_sites = p1.shape[0]
    num_comparisons = p1.shape[1]
    result = []
    for i in range(num_comparisons):
        result.append(np.sum(np.mod(p1+p2, 2))/num_sites)
        p2 = np.roll(p2,1, axis=1)
    return result

In [ ]:
def Fst(ts):
    G = ts.genotype_matrix()
    p1 = G[:,:20]
    p2 = G[:,20:40]
    within = (np.mean(comparison_within(p1)) + np.mean(comparison_within(p2)))/2
    between = np.mean(comparison_between(p1, p2))
    total = np.mean([between, within])
    return (total - within)/total